In [5]:
import librosa
import numpy as np
import os
import pandas as pd
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2Model


In [2]:
emotion_categories = ['NEU', 'HAP', 'SAD', 'ANG', 'FEA', 'DIS']

In [6]:
data_dir = "../data/CREMA-D/"

audio_labels = []
audio_arrays = []

for file in os.listdir(data_dir):
    for emotion in emotion_categories:
        if emotion in file:
            audio_labels.append(emotion)
            audio, sr = librosa.load(os.path.join(data_dir, file), sr=16000)
            audio_arrays.append(audio)

In [8]:
df = pd.DataFrame()
df["audio_array"] = audio_arrays
df["label"] = audio_labels

In [9]:
df.head()

,audio_array,label
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",ANG
1,"[0.00091552734, 0.0013122559, 0.0015869141, 0....",DIS
2,"[0.008026123, 0.008453369, 0.008361816, 0.0074...",FEA
3,"[-0.004333496, -0.0032653809, -0.0030212402, -...",HAP
4,"[0.0024108887, 0.001373291, 0.0013122559, 0.00...",NEU


In [10]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
sample_audio = df['audio_array'][7]

In [23]:
input_values = processor(sample_audio, return_tensors="pt").input_values

It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.


In [24]:
input_values.shape

torch.Size([1, 33100])

In [25]:
hidden_states = model(input_values).last_hidden_state

In [26]:
hidden_states.shape

torch.Size([1, 103, 768])

In [27]:
embeddings = torch.mean(hidden_states, dim=1)
embeddings.shape

torch.Size([1, 768])